In [ ]:
plt.figure(figsize=(10,4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, y0.reshape(-1), label="原结构")
plt.plot(t_s, u1[0, :], label="被动控制系统")
plt.plot(t_s, y2, label="可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$',fontname = 'SimSun',fontsize = 10)
plt.ylabel(r'位移 $(m)$',fontname = 'SimSun',fontsize = 10)
font = {
    "family": "SimSun",
    'weight':'normal',
    "size": 8
    }
plt.legend(prop = font, loc='best')
pic_name = r"位移对比"
plt.savefig(r'./pic/{}.png'.format(pic_name),dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10,4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, dy0.reshape(-1), label="原结构")
plt.plot(t_s, du1[0, :], label="被动控制系统")
plt.plot(t_s, dy2, label="可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$',fontname = 'SimSun',fontsize = 10)
plt.ylabel(r'速度 $(m/s)$',fontname = 'SimSun',fontsize = 10)
font = {
    "family": "SimSun",
    'weight':'normal',
    "size": 8
    }
plt.legend(prop = font, loc='best')
pic_name = r"速度对比"
plt.savefig(r'./pic/{}.png'.format(pic_name),dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10,4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ddy0_1_ab.reshape(-1), label="原结构")
plt.plot(t_s, ddu1_ab[0, :], label="被动控制系统")
plt.plot(t_s, ddy2_ab, label="可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$',fontname = 'SimSun',fontsize = 10)
plt.ylabel(r'加速度 $(m/s^2)$',fontname = 'SimSun',fontsize = 10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight':'normal',
    "size": 8
    }
plt.legend(prop = font, loc='best')
pic_name = r"加速度对比"
plt.savefig(r'./pic/{}.png'.format(pic_name),dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 半主动控制结构
u2_ml = np.zeros([2, wave_len_s])
du2_ml = np.zeros([2, wave_len_s])
du2_ml[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft_ml = np.zeros(wave_len_s)
ud_ml = np.zeros(wave_len_s)
dud_ml = np.zeros(wave_len_s)
ddud_ml = np.zeros(wave_len_s)
fmax_ml = np.zeros(wave_len_s)
fmin_ml = np.zeros(wave_len_s)
fa_ml = np.zeros(wave_len_s)
ud_ml_pre = 0
dud_ml_pre = 0

str_pre_disp = 0
str_pre_vel = 0
str_pre_acc = 0
str_cur_disp = 0
str_cur_vel = 0
str_cur_acc = 0
ft_ml_pre = 0
ft_ml_next = 0

y2_ml = np.zeros(wave_len_s)
dy2_ml = np.zeros(wave_len_s)
ddy2_ml = np.zeros(wave_len_s)
ddy2_ab_ml = np.zeros(wave_len_s)
ag_pre = 0
for i in range(wave_len_s - 1):
    str_cur_disp = u2_ml[0, i - 1]
    str_cur_vel = u2_ml[1, i - 1]
    str_cur_acc = du2_ml[1, i - 1]
    ft_ml_cur = ft_ml[i - 1]
    x = np.array([str_cur_disp, str_cur_vel, str_cur_acc, str_pre_disp, str_pre_vel, str_pre_acc, ft_ml_pre, ft_ml_cur], ndmin=2)
    x = ss.transform(x)
    ft_ml[i] = GBoost.predict(x)[0]
    ft_ml_pre = ft_ml_cur
    str_pre_disp = str_cur_disp
    str_pre_vel = str_cur_vel
    str_pre_acc = str_cur_acc

    ud_ml[i] = u2_ml[0, i] - ft_ml[i] / kd  # 当前惯容位移
    dud_ml[i] = (ud_ml[i] - ud_ml[i - 1]) / dt_s  # 当前惯容速度
    ddud_ml[i] = (dud_ml[i] - dud_ml[i - 1]) / dt_s  # 当前惯容加速度

    # 当前出力上限
    fmax_ml[i] = max(
        [min_max * ddud_ml[i] + cd_max * dud_ml[i],
         min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    # 当前出力下限
    fmin_ml[i] = min([min_max * ddud_ml[i] + cd_max * dud_ml[i],
                   min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    while True:
        if ft_ml[i] < fmin_ml[i]:
            ft_ml[i] = fmin_ml[i]
        elif ft_ml[i] > fmax_ml[i]:
            ft_ml[i] = fmax_ml[i]
        else:
            fa_ml[i] = ft_ml[i]
            break
        ud_ml[i] = u2_ml[0, i] - ft_ml[i] / kd  # 当前惯容位移
        dud_ml[i] = (ud_ml[i] - ud_ml[i - 1]) / dt_s  # 当前惯容速度
        ddud_ml[i] = (dud_ml[i] - dud_ml[i - 1]) / dt_s  # 当前惯容加速度

        # 当前出力上限
        fmax_ml[i] = max(
            [min_max * ddud_ml[i] + cd_max * dud_ml[i],
             min_min * ddud_ml[i] + cd_min * dud_ml[i]])
        # 当前出力下限
        fmin_ml[i] = min([min_max * ddud_ml[i] + cd_max * dud_ml[i],
                       min_min * ddud_ml[i] + cd_min * dud_ml[i]])

    # ud_ml_pre = ud_ml[i]
    # dud_ml_pre = dud_ml[i]
    # Newmark_beta法
    ag = wave_s[i] + fa_ml[i] / m
    z_acc = ag - ag_pre
    z_y = (-m * z_acc + (4 / dt) * m * dy2_ml[i] + 2 * m * ddy2_ml[i] + 2 * c * dy2_ml[i]) / (k + 2 * c / dt + m * 4 / (dt ** 2))
    y2_ml[i + 1] = y2_ml[i] + z_y
    z_dy = 2 / dt * z_y - 2 * dy2_ml[i]
    dy2_ml[i + 1] = dy2_ml[i] + z_dy
    z_ddy = 4 / (dt ** 2) * z_y - (4 / dt) * dy2_ml[i] - 2 * ddy2_ml[i]
    ddy2_ml[i + 1] = ddy2_ml[i] + z_ddy
    ddy2_ab_ml[i] = ddy2_ml[i] + ag

    ag_pre = ag
    u2_ml[0, i + 1] = y2_ml[i + 1]
    u2_ml[1, i + 1] = dy2_ml[i + 1]
    du2_ml[0, i] = dy2_ml[i]
    du2_ml[1, i] = ddy2_ab_ml[i]